In [80]:
import numpy as np
import pandas as pd

In [100]:
df = pd.read_csv("input_power.csv", low_memory = False)
df.head()

,country,country_long,name,gppd_idnr,capacity_mw,latitude,longitude,primary_fuel,other_fuel1,other_fuel2,...,estimated_generation_gwh_2013,estimated_generation_gwh_2014,estimated_generation_gwh_2015,estimated_generation_gwh_2016,estimated_generation_gwh_2017,estimated_generation_note_2013,estimated_generation_note_2014,estimated_generation_note_2015,estimated_generation_note_2016,estimated_generation_note_2017
0,AFG,Afghanistan,Kajaki Hydroelectric Power Plant Afghanistan,GEODB0040538,33.0,32.322,65.1190,Hydro,NaN,NaN,...,123.77,162.90,97.39,137.76,119.50,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1
1,AFG,Afghanistan,Kandahar DOG,WKS0070144,10.0,31.670,65.7950,Solar,NaN,NaN,...,18.43,17.48,18.25,17.70,18.29,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE
2,AFG,Afghanistan,Kandahar JOL,WKS0071196,10.0,31.623,65.7920,Solar,NaN,NaN,...,18.64,17.58,19.10,17.62,18.72,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE
3,AFG,Afghanistan,Mahipar Hydroelectric Power Plant Afghanistan,GEODB0040541,66.0,34.556,69.4787,Hydro,NaN,NaN,...,225.06,203.55,146.90,230.18,174.91,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1
4,AFG,Afghanistan,Naghlu Dam Hydroelectric Power Plant Afghanistan,GEODB0040534,100.0,34.641,69.7170,Hydro,NaN,NaN,...,406.16,357.22,270.99,395.38,350.80,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1


In [108]:
chunks = np.array_split(df, df.shape[0]/500)

In [109]:
len(chunks)

69

In [115]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises", timeout = 10)


def city_state_country(coord):
    try:
        location = geolocator.reverse(coord, exactly_one=True, language = "en")
        if not location:

            # if you see many in a row, it's probably Nominatim blocking you.
            # if it's just every once in a while, there were just some bad results. 
            print('Failed with coord: ', coord)
            city, state, country = None, None, None
            return city, state, country
        address = location.raw['address']
        city = address.get('city', '')
        state = address.get('state', '')
        country = address.get('country', '')
        return city, state, country
#print(city_state_country("30.930508, 75.8419883"))

    except ValueError:
     return(0)


In [117]:
from time import sleep

for i in range(20,len(chunks)):
    chunks[i]['location'] = (chunks[i][['latitude', 'longitude']].astype(str)
                   .apply(lambda row: city_state_country(', '.join(row)),
                          axis=1)
                           )
    print("chunk: ", i)
    sleep(10)

chunk:  20
chunk:  21
chunk:  22
chunk:  23
Failed with coord:  54.5, 7.7999
Failed with coord:  54.36, 5.98
Failed with coord:  54.983, 13.162
Failed with coord:  53.9667, 6.55
Failed with coord:  55.019, 7.774
chunk:  24
Failed with coord:  54.509, 6.367
Failed with coord:  54.041, 6.994
chunk:  25
Failed with coord:  54.392, 7.704
Failed with coord:  55.1416, 7.1945
chunk:  26
chunk:  27
chunk:  28
chunk:  29
chunk:  30
chunk:  31
chunk:  32
chunk:  33
Failed with coord:  54.036, 5.963
Failed with coord:  52.405, 4.1619
chunk:  34
Failed with coord:  52.588, 4.223
Failed with coord:  52.588, 4.223
chunk:  35
chunk:  36
chunk:  37
chunk:  38
Failed with coord:  17.7197, -17.2996
chunk:  39
chunk:  40
Failed with coord:  9.919, 80.4894
chunk:  41
chunk:  42
Failed with coord:  58.0882, -2.9502
chunk:  43
chunk:  44
Failed with coord:  51.88, 2.04
Failed with coord:  51.9176, 1.9284
Failed with coord:  53.6815, 1.4207
chunk:  45
Failed with coord:  51.6217, 1.4958
chunk:  46
chunk:  47

In [112]:
from geopy.geocoders import Nominatim
from time import sleep

geolocator = Nominatim(user_agent="geoapiExercises", timeout = 10)

df_power_org = pd.read_csv('global_power_plant_database.csv', low_memory=False)

df_power_org = df_power_org[df_power_org.longitude.notnull()]
df_power_org = df_power_org[df_power_org.latitude.notnull()]

def city_state_country(row):
    coord = f"{row['latitude']}, {row['longitude']}"
    sleep(1)
    location = geolocator.reverse(coord, exactly_one=True, language='en')
    if not location:
        # if you see many in a row, it's probably Nominatim blocking you.
        # if it's just every once in a while, there were just some bad results. 
        print('Failed with coord: ', coord)
        row['city'], row['state'], row['country2'] = None, None, None
        return row
    address = location.raw['address']
    city = address.get('city', '')
    state = address.get('state', '')
    country = address.get('country', '')
    row['city'] = city
    row['state'] = state
    row['country2'] = country
    return row

df_power_org = df_power_org.apply(city_state_country, axis=1)

Failed with coord:  51.66, 2.8
Failed with coord:  51.6186, 2.8998
Failed with coord:  51.556, 2.9689
Failed with coord:  51.54, 2.921


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=-28.9238&lon=-49.6741&format=json&accept-language=en&addressdetails=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x17ab67eb0>: Failed to establish a new connection: [Errno 65] No route to host'))

In [133]:
appended_data = pd.DataFrame()

for i in range(len(chunks)):
    appended_data = pd.concat([appended_data,chunks[i]])
    

In [134]:
appended_data.shape

(34936, 37)

In [136]:
appended_data.to_csv("output.csv")

In [145]:
appended_data[['city', 'state', 'country']] = appended_data['location'].str.split(',',expand=True)

ValueError: Columns must be same length as key

In [144]:
appended_data.head()

,country,country_long,name,gppd_idnr,capacity_mw,latitude,longitude,primary_fuel,other_fuel1,other_fuel2,...,estimated_generation_gwh_2015,estimated_generation_gwh_2016,estimated_generation_gwh_2017,estimated_generation_note_2013,estimated_generation_note_2014,estimated_generation_note_2015,estimated_generation_note_2016,estimated_generation_note_2017,location,"(city, state, country)"
0,AFG,Afghanistan,Kajaki Hydroelectric Power Plant Afghanistan,GEODB0040538,33.0,32.322,65.1190,Hydro,NaN,NaN,...,97.39,137.76,119.50,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,"(, Helmand Province, Afghanistan)",NaN
1,AFG,Afghanistan,Kandahar DOG,WKS0070144,10.0,31.670,65.7950,Solar,NaN,NaN,...,18.25,17.70,18.29,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,"(Aino Maina, Kandahar Province, Afghanistan)",NaN
2,AFG,Afghanistan,Kandahar JOL,WKS0071196,10.0,31.623,65.7920,Solar,NaN,NaN,...,19.10,17.62,18.72,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,"(Aino Maina, Kandahar Province, Afghanistan)",NaN
3,AFG,Afghanistan,Mahipar Hydroelectric Power Plant Afghanistan,GEODB0040541,66.0,34.556,69.4787,Hydro,NaN,NaN,...,146.90,230.18,174.91,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,"(, Kabul Province, Afghanistan)",NaN
4,AFG,Afghanistan,Naghlu Dam Hydroelectric Power Plant Afghanistan,GEODB0040534,100.0,34.641,69.7170,Hydro,NaN,NaN,...,270.99,395.38,350.80,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,"(Surobi, Kabul Province, Afghanistan)",NaN
